## Scotch

The scraping notebook walked through some basic web scraping with html that was fairly straightforward and well behaved to show you the basics of the involved libraries. And there are a lot of nicely written tutorials on the web where you can see a polished version of what people did for scraping. But the question I was struggling with while putting this together was how do you really teach someone to scrape? And what is a good site to teach with?

While I was thinking about it, I recieved the weekly email from one of my favorite online retailers, [Drink Up NY](http://www.drinkupny.com/Default.asp). Since I am quite fond of their inventory and I had scraping on the brain, I thought I would see if I could scrape the data (and I didn't see any good reason [why I shouldn't](http://www.drinkupny.com/robots.txt)). As I was playing around with this notebook I thought it might be a good way to demonstrate a thought process for approaching scraping. This is not a definitive way to scrape, it is just the way *I* approached *this* page sitting around on my couch the week before this tutorial. Like anything in Python, there are a lot of ways to accomplish something. 

I can read and write some html and I have broken some CSS on WordPress before, but I am not a web dev. I would call myself barely competent in that realm. If I were, I imagine this would go a lot faster sometimes. I do like tinkering around and trying things though, so that is how I approach scraping and what I will demonstrate here.

So, let's see what is going on with Scotch prices on Drink Up NY!

In [ ]:
import requests
from bs4 import BeautifulSoup

I am using [requests](http://docs.python-requests.org/en/master/) here, but I could have just as easily used [urllib](https://docs.python.org/3.5/library/urllib.html). 

First, we need to get the page. If you navigate around the [Drink Up NY](http://www.drinkupny.com/Default.asp) site a bit, you can see they have a lot of drop down menus. I am just intereted in the [Scotch](https://www.drinkupny.com/single-malt-scotch-whisky-s/77.htm) page right now. I navigated here through the "SPIRITS ETC" - "Whisk(e)y" - "Scotch Whisky" links. For scraping, I am going to change the "per page" option to "120 per page" so I have everything on one page.

In [ ]:
html_file = "https://www.drinkupny.com/single-malt-scotch-whisky-s/77.htm?searching=Y&sort=9&cat=77&show=120&page=1"
html_rpt = requests.get(html_file)

In [ ]:
if html_rpt.status_code == 200:
    print(html_rpt.content)
else:
    print(html_rpt.status_code)

It worked. Let's parse and prettify it.

In [ ]:
soup = BeautifulSoup(html_rpt.content, 'html.parser')
print(soup.prettify())

Now let's take a look through the structure and find the data. 

In [ ]:
# I wouldn't expect my data to be here, but there is some metadata if we want it.
soup.head

In [ ]:
soup.body

We can certainly pull a lot of links easily with their navigation system. Here is one way.

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

It looks like there are a few tables in there as well.

In [ ]:
soup.body.table

In [ ]:
len(soup.find_all('table'))

The Scotch data I want is probably in one of those 12 tables. 

In [ ]:
soup.find_all('table')[0]

Nope, that's not it.

In [ ]:
soup.find_all('table')[1]

Nope, not there either.

In [ ]:
soup.find_all('table')[2]

BINGO! Where we have tables, we have rows. 

In [ ]:
soup.find_all('table')[2].tr

That wasn't overly useful. We do have some nice looking CSS selector options though like this:
```<div class="v-product">```

In [ ]:
len(soup.select(".v-product"))

65 items for that selector. And 65 bottles on the page. That could be it. 

When we use select, Beautiful Soup gives us a list. So let's look at the list elements.

In [ ]:
soup.select(".v-product")[0]

So when we use the CSS selector, we get a list where each item in the list is one of our scotch listings. How can we pull some meaning full data from that? I would like to get the name, link to the product, and the pricing info. Let's get one item to work with so we can try to parse this out. 

In [ ]:
table = soup.select(".v-product")

In [ ]:
table[0]

In [ ]:
table[1]

In [ ]:
scotch = table[0]
scotch

In [ ]:
scotch.a

In [ ]:
scotch.a.string

In [ ]:
scotch.a.text

None of that is particularly useful. Let's try working with the CSS selectors again.

In [ ]:
scotch.select(".v-product__title")

In [ ]:
scotch.select(".v-product__title").text

Ooops. Remember select returns a list.

In [ ]:
scotch.select(".v-product__title")[0].text

Now we are getting somewhere.

In [ ]:
scotch.select(".product_productprice")[0]

In [ ]:
scotch.select(".product_productprice")[0].text

In [ ]:
scotch.select(".product_saleprice")[0].text

In [ ]:
scotch.select(".product_listprice")[0].text

Now what about the product link?

In [ ]:
scotch.select(".v-product__title")[0].get('href')

That's it!  [https://www.drinkupny.com/Aberlour-a-bunadh-p/s0958.htm](https://www.drinkupny.com/Aberlour-a-bunadh-p/s0958.htm)

Now we can manipulate our strings a bit to strip off the parts we don't want.

In [ ]:
scotch.select(".v-product__title")[0].text.split('\n', 1)[1]

In [ ]:
scotch.select(".v-product__title")[0].text.split('\n', 1)[1].strip()

In [ ]:
scotch.select(".v-product__title")[0].text.strip()

In [ ]:
scotch.select(".product_productprice")[0].text.split('$', 1)[1]

In [ ]:
scotch.select(".product_productprice")[0].text.split('$', 1)[1].strip()

In [ ]:
scotch.select(".product_saleprice")[0].text.split('$', 1)[1].strip()

In [ ]:
scotch.select(".product_listprice")[0].text.split('$', 1)[1].strip()

Let's pull it all together and see if it works for parsing all of the data.

In [ ]:
len(table)

In [ ]:
import pandas as pd

In [ ]:
i = 0
column_names = ['name', 'url', 'regular_price', 'our_price', 'sale_price']

df = pd.DataFrame(columns = column_names)

In [ ]:
while i < len(table):
    new_scotch = pd.Series([table[i].select(".v-product__title")[0].text.strip(), \
                                 table[i].select(".v-product__title")[0].get('href'), \
                                 table[i].select(".product_listprice")[0].text.split('$', 1)[1], \
                                 table[i].select(".product_productprice")[0].text.split('$', 1)[1], \
                                 table[i].select(".product_saleprice")[0].text.split('$', 1)[1]], column_names)
    df = df.append(new_scotch, ignore_index = True)
    i += 1

No errors, so that is a good sign. Can we now can easily compare Scotch prices?

In [ ]:
df

If you went through this notebook instead of trying a page you wanted to scrape, I fully expect you to go try and scrape your own page now!

If you are interested in working with [scrapy](https://doc.scrapy.org/en/latest/index.html), I highly recommend starting with the [tutorial](https://doc.scrapy.org/en/latest/intro/tutorial.html). I have provided a [scrapy sample](./scrapy/scotch) that uses scrapy to begin pulling data from the site utilized here.